In [3]:
import pandas as pd
import requests
from config import settings

In [19]:
print(dir(settings))

['Config', '__abstractmethods__', '__annotations__', '__class__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_repr__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_forward_refs__', '__validators__', '_abc_impl', '_build_values', '_calculate_keys', '_copy_and_set_values', '_decompose_class', '_enforce_dict_if_root', '_get_value', '_init_

In [20]:
settings.db_name

'stocks.sqlite'

In [21]:
settings.alpha_api_key

'FQRMMQ7N7O6RSMY3'

In [22]:
settings.model_directory

'models'

In [4]:
#Build the api url XLK S&P Technology Select Sector
ticker = "XLRE"
outputsize = "compact"
datatype = "json"

url = ("https://www.alphavantage.co/query?"
        "function=TIME_SERIES_DAILY_ADJUSTED&"
        f"symbol={ticker}&"
        f"datatype={datatype}&"
        f"outputsize={outputsize}&"
        f"apikey={settings.alpha_api_key}"
        )
print(url)

https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=XLRE&datatype=json&outputsize=compact&apikey=FQRMMQ7N7O6RSMY3


In [5]:
response = requests.get(url=url)
print("Response type:", type(response))

Response type: <class 'requests.models.Response'>


In [6]:
response_data = response.json()
response_data.keys()

dict_keys(['Meta Data', 'Time Series (Daily)'])

In [7]:
stock_data = response_data["Time Series (Daily)"]

In [9]:
list(stock_data.items())[:3]

[('2023-01-04',
  {'1. open': '37.4',
   '2. high': '38.26',
   '3. low': '37.27',
   '4. close': '37.92',
   '5. adjusted close': '37.92',
   '6. volume': '7586940',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0'}),
 ('2023-01-03',
  {'1. open': '37.23',
   '2. high': '37.56',
   '3. low': '36.61',
   '4. close': '37.07',
   '5. adjusted close': '37.07',
   '6. volume': '7060342',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0'}),
 ('2022-12-30',
  {'1. open': '37.04',
   '2. high': '37.18',
   '3. low': '36.53',
   '4. close': '36.93',
   '5. adjusted close': '36.93',
   '6. volume': '5087853',
   '7. dividend amount': '0.0000',
   '8. split coefficient': '1.0'})]

In [50]:
stock_data["2023-01-04"]

{'1. open': '37.4',
 '2. high': '38.26',
 '3. low': '37.27',
 '4. close': '37.92',
 '5. adjusted close': '37.92',
 '6. volume': '7586940',
 '7. dividend amount': '0.0000',
 '8. split coefficient': '1.0'}

In [51]:
df = pd.DataFrame.from_dict(stock_data, orient="index", dtype=float)

In [52]:
df.index = pd.to_datetime(df.index)
df.index.name = "date"

In [53]:
df.columns = [c.split(". ")[1] for c in df.columns] 


In [54]:
df.head()

,open,high,low,close,adjusted close,volume,dividend amount,split coefficient
date,,,,,,,,
2023-01-04,37.400,38.260,37.270,37.92,37.92,7586940.0,0.0,1.0
2023-01-03,37.230,37.560,36.610,37.07,37.07,7060342.0,0.0,1.0
2022-12-30,37.040,37.180,36.530,36.93,36.93,5087853.0,0.0,1.0
2022-12-29,36.721,37.365,36.620,37.27,37.27,4143152.0,0.0,1.0
2022-12-28,37.140,37.340,36.425,36.46,36.46,4730881.0,0.0,1.0


In [55]:
df = df.drop(["adjusted close", "volume", "dividend amount", "split coefficient"], axis = 1)

In [56]:
type(df)

pandas.core.frame.DataFrame

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2023-01-04 to 2022-08-12
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    100 non-null    float64
 1   high    100 non-null    float64
 2   low     100 non-null    float64
 3   close   100 non-null    float64
dtypes: float64(4)
memory usage: 3.9 KB


In [58]:
def get_historical_data(ticker, output_size="full"):
    """
    A function that downloads the historical data from Alpha Vantage website using free API key.

    Parameters required:
    -------------------
    Ticker - The stock symbol
    Output_size  - The two options available are full or compact. The function is set to full by default.
    """

    url = ("https://www.alphavantage.co/query?"
            "function=TIME_SERIES_DAILY_ADJUSTED&"
            f"symbol={ticker}&"
            f"outputsize={output_size}&"
            f"apikey={settings.alpha_api_key}"
         )
    response = requests.get(url=url) 
    #Convert the response to json
    response_json = response.json()
    #Extract the Daily Time series
    data_dict = response_json["Time Series (Daily)"]
    #Convert the data dictionary into DataFrame
    df = pd.DataFrame.from_dict(data_dict, orient = "index", dtype = float)
    #Convert the index to datetime and set the name to date
    df.index = pd.to_datetime(df.index)
    df.index.name = "date"
    #Clean the column names
    df.columns = [c.split(". ")[1] for c in df.columns] 
    #Drop the unwanted columns
    df = df.drop(["adjusted close", "volume", "dividend amount", "split coefficient"], axis = 1)
    return df

In [59]:
xlk_data = get_historical_data(ticker="XLK", output_size="compact")

print(xlk_data.info())
xlk_data.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2023-01-04 to 2022-08-12
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    100 non-null    float64
 1   high    100 non-null    float64
 2   low     100 non-null    float64
 3   close   100 non-null    float64
dtypes: float64(4)
memory usage: 3.9 KB
None


,open,high,low,close
date,,,,
2023-01-04,123.90,124.37,122.220,123.58
2023-01-03,125.77,126.60,122.380,123.26
2022-12-30,123.32,124.51,122.550,124.44
2022-12-29,122.62,125.03,122.605,124.61
2022-12-28,123.12,124.27,121.290,121.43


In [2]:
from data import AlphaVantageApi

api = AlphaVantageApi()
xlk_data = api.get_historical_data(ticker="XLK")
print(xlk_data.info())
xlk_data.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5832 entries, 2023-01-04 to 1999-11-01
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    5832 non-null   float64
 1   high    5832 non-null   float64
 2   low     5832 non-null   float64
 3   close   5832 non-null   float64
dtypes: float64(4)
memory usage: 227.8 KB
None


,open,high,low,close
date,,,,
2023-01-04,123.90,124.37,122.220,123.58
2023-01-03,125.77,126.60,122.380,123.26
2022-12-30,123.32,124.51,122.550,124.44
2022-12-29,122.62,125.03,122.605,124.61
2022-12-28,123.12,124.27,121.290,121.43
